In [ ]:
import pandas as pd

!pip install pyathena
from pyathena import connect

import json


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 112.2/112.2 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.6/140.6 kB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.5/14.5 MB 103.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 6.8 MB/s eta 0:00:00


In [ ]:
from datetime import datetime, timedelta
hoy = datetime.now()             # timestamp actual
hace_2_semanas = hoy - timedelta(weeks=2)

print("Hoy:", hoy)
print("Hace 2 semanas:", hace_2_semanas)

ymd_2_semanas = hace_2_semanas.strftime("%Y-%m-%d")
print(ymd_2_semanas)

Hoy: 2025-12-03 15:01:46.084607
Hace 2 semanas: 2025-11-19 15:01:46.084607
2025-11-19


In [ ]:
# Colores ANSI
class C:
    RED     = '\033[91m'
    GREEN   = '\033[92m'
    YELLOW  = '\033[93m'
    BLUE    = '\033[94m'
    BOLD    = '\033[1m'
    END     = '\033[0m'
    ORANGE  = '\033[38;5;208m'   # Naranja (256-colors)


# print(f"{C.BOLD}{C.BLUE}=== INFORME MENSUAL ==={C.END}")
# print(f"{C.BLUE}=== INFORME MENSUAL ==={C.END}")
# print(f"{C.GREEN}✔ Procesado correctamente{C.END}")
# print(f"{C.YELLOW}⚠ Advertencia: valores faltantes detectados{C.END}")
# print(f"{C.RED}✘ Error en la carga de datos{C.END}")


In [ ]:
#%% Credenciales de AmazonAthena

with open(r"/content/credenciales actualizado.txt") as f:
    creds = json.load(f)

conn = connect(
    aws_access_key_id     = creds["AccessKeyId"],
    aws_secret_access_key = creds["SecretAccessKey"],
    aws_session_token     = creds["SessionToken"],
    s3_staging_dir        = creds["s3_staging_dir"],
    region_name           = creds["region_name"]

    )


In [ ]:
# Moneda de la operación distinta en fac_requests y hubspot__deal

query ='''
with moneda_fr as  (

SELECT
    FR.CODE,
    FR.proforma_simulation_currency as moneda_fr

FROM prod_datalake_analytics.FAC_REQUESTS AS FR
where FR.status not in ('rejected', 'canceled')

),

moneda_hd as (

select
    dealname,
    moneda_del_monto_financiado as moneda_hd
from prod_datalake_master.hubspot__deal
where pipeline = '14026011'
and dealstage not in ('14026016', '14026018', '1105313628')

)

select
    fr.code,
    fr.moneda_fr,
    hd.moneda_hd

from moneda_fr as fr
left join moneda_hd as hd
on lower(fr.code) = lower(hd.dealname)
where fr.moneda_fr <> hd.moneda_hd

'''

cursor = conn.cursor()
cursor.execute(query)

# Obtener los resultados
resultados = cursor.fetchall()

# Obtener los nombres de las columnas
column_names = [desc[0] for desc in cursor.description]

# Convertir los resultados a un DataFrame de pandas
df_moneda = pd.DataFrame(resultados, columns = column_names)



In [ ]:
# RUCs de proveedor o deudor que acaban en “.com”

query ='''
select
    dealname,
    ruc_proveedor,
    ruc_cliente

from prod_datalake_master.hubspot__deal

where pipeline = '14026011'
and lower(ruc_proveedor) like lower('%.com%')
or lower(ruc_cliente) like lower('%.com%')

'''

cursor = conn.cursor()
cursor.execute(query)

# Obtener los resultados
resultados = cursor.fetchall()

# Obtener los nombres de las columnas
column_names = [desc[0] for desc in cursor.description]

# Convertir los resultados a un DataFrame de pandas
df_rucs_dotcom = pd.DataFrame(resultados, columns = column_names)

In [ ]:
# Casos urgente de fechas de desembolso diferentes entre Hubspot y Fac_requests, porque cambian el mes

query ='''
with fechas_desembolso_request as (

select
    code,
     cast ( date_add('hour', -5, interest_proforma_disbursement_date)  as date) as f_desembolso
from prod_datalake_analytics.fac_requests
where status not in ('rejected', 'canceled')

), fechas_desembolso_hubspot as (

select
    dealname as code,
    fecha_de_desembolso__factoring_ as f_desembolso
from prod_datalake_master.hubspot__deal
where pipeline = '14026011'
and dealstage not in ('14026016', '14026018', '1105313628')

)
select
    fdr.code,
    fdr.f_desembolso as f_desembolso_request,
    fdh.f_desembolso as f_desembolso_hubspot

from  fechas_desembolso_request as fdr
left join fechas_desembolso_hubspot as fdh
on lower(fdr.code) = lower(fdh.code)

where date_format(fdr.f_desembolso, '%Y%m') <> date_format(fdh.f_desembolso, '%Y%m')
and fdr.f_desembolso >= date '2025-01-01'

order by fdr.f_desembolso desc

'''

cursor = conn.cursor()
cursor.execute(query)

# Obtener los resultados
resultados = cursor.fetchall()

# Obtener los nombres de las columnas
column_names = [desc[0] for desc in cursor.description]

# Convertir los resultados a un DataFrame de pandas
df_fecha_desembolso = pd.DataFrame(resultados, columns = column_names)

In [ ]:
# Subasta Activa, más de 2 semanas sin cambios:

query =f'''
select
    dealname,
    fecha_de_desembolso__factoring_,
    'Subasta Activa' as etapa_negocio
from prod_datalake_master.hubspot__deal
where dealstage = '14026014'
and fecha_de_desembolso__factoring_ is not null
and fecha_de_desembolso__factoring_ < date '{ymd_2_semanas}'
order by fecha_de_desembolso__factoring_

'''

cursor = conn.cursor()
cursor.execute(query)

# Obtener los resultados
resultados = cursor.fetchall()

# Obtener los nombres de las columnas
column_names = [desc[0] for desc in cursor.description]

# Convertir los resultados a un DataFrame de pandas
df_s_activa = pd.DataFrame(resultados, columns = column_names)

In [ ]:
# Información de Subasta Pendiente, más de 2 semanas sin cambios:

query =f'''
select
    dealname,
    fecha_de_desembolso__factoring_,
    'Información de Subasta pendiente' as etapa_negocio
from prod_datalake_master.hubspot__deal
where dealstage = '14026013'
and fecha_de_desembolso__factoring_ is not null
and fecha_de_desembolso__factoring_ < date '{ymd_2_semanas}'
order by fecha_de_desembolso__factoring_

'''

cursor = conn.cursor()
cursor.execute(query)

# Obtener los resultados
resultados = cursor.fetchall()

# Obtener los nombres de las columnas
column_names = [desc[0] for desc in cursor.description]

# Convertir los resultados a un DataFrame de pandas
df_info_pendiente = pd.DataFrame(resultados, columns = column_names)

In [ ]:
print(f"{C.BOLD}{C.GREEN}✔ INFORME DE ALERTAS DETECTADAS:{C.END}")
print(f"=============================================================\n")
if df_moneda.shape[0] == 0:
  print(f"{C.GREEN}✔ Sin diferencias entre monedas{C.END}")
else:
  print(f"{C.ORANGE}⚠ Advertencia: Diferencia entre monedas{C.END}")
  print(df_moneda)
print(f"=============================================================\n")
if df_rucs_dotcom.shape[0] == 0:
  print(f"{C.GREEN}✔ Rucs correctos{C.END}")
else:
  print(f"{C.YELLOW}⚠ Advertencia: Formato de rucs incorrecto, corregir en Hubspot{C.END}")
  print(df_rucs_dotcom)
print(f"=============================================================\n")
if df_fecha_desembolso.shape[0] == 0:
  print(f"{C.GREEN}✔ Sin diferencias relevantes entre fechas de desembolso")
else:
  print(f"{C.RED}⚠ Advertencia verificar: Diferencia entre fechas de desembolso{C.END}")
  print(df_fecha_desembolso)
print(f"=============================================================\n")
if df_s_activa.shape[0] == 0:
  print(f"{C.GREEN}✔ Sin subastas activas más de 2 semanas{C.END}")
else:
  print(f"{C.YELLOW}⚠ Advertencia: Consultar subasta en operaciones, no debería estar activa tanto tiempo{C.END}")
  print(df_s_activa)
print(f"=============================================================\n")
if df_info_pendiente.shape[0] == 0:
  print(f"{C.GREEN}✔ Sin Subastas con información pendiente más de 2 semanas{C.END}")
else:
  print(f"{C.YELLOW}⚠ Advertencia: Consultar subasta en operaciones, no debería estar con información pendiente tanto tiempo{C.END}")
  print(df_info_pendiente)
print(f"=============================================================\n")




✔ INFORME DE ALERTAS DETECTADAS:

⚠ Advertencia: Diferencia entre monedas
       code moneda_fr moneda_hd
0  u0BYubI7       PEN       USD
1  LEy90TxT       USD       PEN
2  yQb0mbHC       USD       PEN

✔ Rucs correctos

⚠ Advertencia verificar: Diferencia entre fechas de desembolso
        code f_desembolso_request f_desembolso_hubspot
0   kIl6eI0Z           2025-11-29           2025-12-01
1   0UTw0iKx           2025-10-31           2025-11-03
2   7I1F0WOf           2025-10-09           2025-11-03
3   oFRS0Tjs           2025-09-30           2025-10-06
4   YkbPeN0q           2025-09-22           2025-08-22
5   wYq7OsP3           2025-08-29           2025-09-03
6   iDvsQp0h           2025-08-29           2025-09-03
7   itw43mSi           2025-08-29           2025-09-29
8   r5oHheUw           2025-08-25           2025-09-02
9   7RgNnwlY           2025-08-22           2025-09-01
10  hKF65IAq           2025-07-31           2025-08-13
11  QbLGAyr4           2025-07-30           2025-08-04
1